In [6]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [7]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [8]:
tf.keras.backend.clear_session()

In [9]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [14]:
subject = 'Alagoas - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [15]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_AL.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data

,Ano,Alagoas - Desemprego,Alagoas - IDH Educacao,Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado,Alagoas - IDH Longevidade,Alagoas - IDH Renda,Alagoas - Produção de Cimento (t),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Alagoas - Consumo de Cimento (t)
0,2008,8.543035,0.601766,2.751323e+07,1.369962e+06,7.684387,2.402206e+07,0.749235,0.622573,494.115531,...,2.868108,11.760983,19.541805,1004.903885,3228.342662,26.070097,20.787810,74.407297,3.079016,446.335000
1,2009,8.571316,0.606396,2.770792e+07,1.372464e+06,7.689682,2.403833e+07,0.751769,0.624024,565.545675,...,2.305885,9.472585,20.736709,1004.528048,1611.970867,25.643630,21.162947,77.670858,2.815837,471.072000
2,2010,8.539868,0.610714,2.842678e+07,1.369925e+06,7.728365,2.421845e+07,0.753767,0.625020,637.418103,...,1.958246,9.332485,19.768388,1004.613084,1481.706116,25.439288,21.037877,77.914366,2.782447,583.772000
3,2011,8.526852,0.614717,2.911152e+07,1.363070e+06,7.768204,2.438475e+07,0.755240,0.625568,683.859991,...,2.079088,10.985689,18.126355,1004.876381,1381.865715,25.362543,20.922266,77.670210,2.776959,742.881000
4,2012,12.945391,0.618592,2.970505e+07,1.349554e+06,7.784639,2.449458e+07,0.756056,0.625434,728.643375,...,2.183282,8.159014,17.184340,1005.022203,1424.662145,25.342566,20.944017,77.821942,2.740248,844.864000
5,2013,13.577910,0.621225,3.023816e+07,1.337956e+06,7.798417,2.459006e+07,0.756644,0.625304,726.048009,...,2.403445,7.913561,17.252127,1005.229473,1469.493466,25.323659,21.009855,78.181963,2.678582,791.375000
6,2014,14.169213,0.625244,3.070999e+07,1.324714e+06,7.820611,2.467763e+07,0.756698,0.624996,650.702685,...,2.748397,10.350442,16.874158,1005.478116,1490.005745,25.397446,21.064696,78.095570,2.657262,818.645143
7,2015,14.659464,0.627431,3.107611e+07,1.310803e+06,7.837546,2.473871e+07,0.756172,0.623329,525.071908,...,2.501529,12.454398,16.443616,1005.522737,1495.198000,25.490999,21.080729,77.734091,2.687007,676.300114
8,2016,14.776623,0.643045,3.127251e+07,1.307005e+06,7.840584,2.474349e+07,0.767870,0.631369,350.210234,...,2.073464,13.111908,16.926315,1005.453192,1487.897624,25.451016,21.102972,78.001380,2.705435,538.586743
9,2017,14.347557,0.643776,3.135879e+07,1.302182e+06,7.848958,2.475540e+07,0.767131,0.630858,239.571638,...,2.177386,9.479812,17.228816,1005.412538,1488.339609,25.438238,21.210017,78.555457,2.646980,362.791000


In [16]:
input_data = data.iloc[:-2, 1:-1]
input_data = input_data.drop('Alagoas - Produção de Cimento (t)',axis=1)
input_data

,Alagoas - Desemprego,Alagoas - IDH Educacao,Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado,Alagoas - IDH Longevidade,Alagoas - IDH Renda,Alagoas - IDH,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,8.543035,0.601766,2.751323e+07,1.369962e+06,7.684387,2.402206e+07,0.749235,0.622573,0.655354,5.483335e+08,...,-7723.574973,2.868108,11.760983,19.541805,1004.903885,3228.342662,26.070097,20.787810,74.407297,3.079016
1,8.571316,0.606396,2.770792e+07,1.372464e+06,7.689682,2.403833e+07,0.751769,0.624024,0.658104,5.302764e+08,...,-8961.781709,2.305885,9.472585,20.736709,1004.528048,1611.970867,25.643630,21.162947,77.670858,2.815837
2,8.539868,0.610714,2.842678e+07,1.369925e+06,7.728365,2.421845e+07,0.753767,0.625020,0.660450,5.046425e+08,...,-9895.682385,1.958246,9.332485,19.768388,1004.613084,1481.706116,25.439288,21.037877,77.914366,2.782447
3,8.526852,0.614717,2.911152e+07,1.363070e+06,7.768204,2.438475e+07,0.755240,0.625568,0.662401,4.854622e+08,...,-9989.170714,2.079088,10.985689,18.126355,1004.876381,1381.865715,25.362543,20.922266,77.670210,2.776959
4,12.945391,0.618592,2.970505e+07,1.349554e+06,7.784639,2.449458e+07,0.756056,0.625434,0.663873,4.867661e+08,...,-9429.326328,2.183282,8.159014,17.184340,1005.022203,1424.662145,25.342566,20.944017,77.821942,2.740248
5,13.577910,0.621225,3.023816e+07,1.337956e+06,7.798417,2.459006e+07,0.756644,0.625304,0.664946,5.225848e+08,...,-7157.441850,2.403445,7.913561,17.252127,1005.229473,1469.493466,25.323659,21.009855,78.181963,2.678582
6,14.169213,0.625244,3.070999e+07,1.324714e+06,7.820611,2.467763e+07,0.756698,0.624996,0.666358,5.676413e+08,...,-3453.461327,2.748397,10.350442,16.874158,1005.478116,1490.005745,25.397446,21.064696,78.095570,2.657262
7,14.659464,0.627431,3.107611e+07,1.310803e+06,7.837546,2.473871e+07,0.756172,0.623329,0.666388,5.179392e+08,...,-1442.919387,2.501529,12.454398,16.443616,1005.522737,1495.198000,25.490999,21.080729,77.734091,2.687007
8,14.776623,0.643045,3.127251e+07,1.307005e+06,7.840584,2.474349e+07,0.767870,0.631369,0.678051,4.524144e+08,...,-3128.377228,2.073464,13.111908,16.926315,1005.453192,1487.897624,25.451016,21.102972,78.001380,2.705435
9,14.347557,0.643776,3.135879e+07,1.302182e+06,7.848958,2.475540e+07,0.767131,0.630858,0.677874,4.213689e+08,...,-5386.337108,2.177386,9.479812,17.228816,1005.412538,1488.339609,25.438238,21.210017,78.555457,2.646980


In [17]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     471.072000
1     583.772000
2     742.881000
3     844.864000
4     791.375000
5     818.645143
6     676.300114
7     538.586743
8     362.791000
9     340.278000
10    395.359000
11    581.479000
12    576.578000
13    547.622000
14           NaN
Name: Alagoas - Consumo de Cimento (t), dtype: float64

In [18]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Alagoas - Desemprego,Alagoas - IDH Educacao,Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado,Alagoas - IDH Longevidade,Alagoas - IDH Renda,Alagoas - IDH,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-1.394011,-1.562212,-1.771772,1.267308,-1.824617,-1.828417,-1.530739,-1.310765,-1.525249,1.204752,...,-0.569731,2.026352,0.984445,1.186951,-0.595151,3.297433,1.414461,-1.989133,-2.084147,2.701872
1,-1.383296,-1.252696,-1.630985,1.357232,-1.734035,-1.767156,-1.104764,-0.842247,-1.175856,0.820657,...,-0.838980,0.085558,-0.051710,1.903618,-1.755575,-0.042015,0.116552,0.600096,0.287340,0.438290
2,-1.395211,-0.964094,-1.111174,1.266004,-1.072298,-1.088666,-0.769045,-0.520929,-0.877896,0.275392,...,-1.042057,-1.114490,-0.115145,1.322848,-1.493020,-0.311144,-0.505344,-0.263151,0.464287,0.151102
3,-1.400143,-0.696569,-0.616035,1.019651,-0.390788,-0.462241,-0.521550,-0.344017,-0.630022,-0.132596,...,-1.062386,-0.697344,0.633403,0.338007,-0.680070,-0.517416,-0.738908,-1.061109,0.286869,0.103900
4,0.274014,-0.437536,-0.186847,0.533959,-0.109633,-0.048522,-0.384411,-0.387101,-0.443003,-0.104860,...,-0.940648,-0.337667,-0.646476,-0.226984,-0.229833,-0.428998,-0.799707,-0.910979,0.397127,-0.211851
5,0.513671,-0.261567,0.198648,0.117149,0.126049,0.311122,-0.285597,-0.429212,-0.306719,0.657046,...,-0.446625,0.422337,-0.757614,-0.186328,0.410129,-0.336376,-0.857248,-0.456556,0.658738,-0.742231
6,0.737712,0.007045,0.539832,-0.358708,0.505729,0.641003,-0.276579,-0.528629,-0.127318,1.615453,...,0.358807,1.613108,0.345772,-0.413022,1.177832,-0.293997,-0.632685,-0.078042,0.595960,-0.925602
7,0.923465,0.153222,0.804578,-0.858614,0.795419,0.871076,-0.364842,-1.066695,-0.123527,0.558230,...,0.796000,0.760923,1.298413,-0.671248,1.315604,-0.283270,-0.347965,0.032622,0.333289,-0.669771
8,0.967856,1.196863,0.946595,-0.995086,0.847385,0.889099,1.600990,1.528559,1.358195,-0.835563,...,0.429496,-0.716758,1.596124,-0.381740,1.100879,-0.298352,-0.469649,0.186147,0.527516,-0.511267
9,0.805286,1.245745,1.008989,-1.168405,0.990653,0.933950,1.476783,1.363339,1.335691,-1.495939,...,-0.061498,-0.358019,-0.048438,-0.200310,0.975357,-0.297439,-0.508538,0.924979,0.930139,-1.014033


In [19]:
reshaped_train_input = dfToInputRNN(train_input)

In [20]:
validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
validation_input = dfToInputRNN(validation_input)
validation_input

array([[[ 0.61011402,  1.1470101 ,  0.8305478 , -0.97835297,
          0.87090648,  0.72017857,  0.90277134,  1.10319335,
          1.10753973, -1.0453709 ,  1.21859064, -0.99976018,
          0.79428371,  1.15575749, -0.52559858,  1.30260559,
         -0.55253046, -0.25130359,  1.38787176,  1.47154451,
         -1.80649616,  1.09296291,  0.94281138,  0.52836084,
          0.61258103,  0.87656257,  1.47801536,  1.71982086,
          1.28221181,  1.39086472,  1.58209844,  1.08872091,
          0.55212962, -1.81398118,  1.44113674,  1.11191094,
          1.30470835,  0.86390516, -1.67365968, -1.70738939,
         -1.55813196, -0.48498301, -0.75714639,  0.40770893,
         -0.27394734,  2.02323219,  1.2908951 ,  1.70465513,
          1.81148084,  1.39968653, -1.36191442,  1.28024593,
          1.11054795,  0.12662799,  1.40414112,  1.40468999,
          1.45643959,  1.58509336,  1.32283157,  1.30222047,
          1.55138602,  1.91801602, -0.77489784,  2.32098884,
         -1.1942541 , -1

In [21]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0     471.072000
1     583.772000
2     742.881000
3     844.864000
4     791.375000
5     818.645143
6     676.300114
7     538.586743
8     362.791000
9     340.278000
10    395.359000
11    581.479000
Name: Alagoas - Consumo de Cimento (t), dtype: float64

In [22]:
# Alvo para validação do treinamento
validation_target = target_data.iloc[split_index - 1:-3]
validation_target

11    581.479
Name: Alagoas - Consumo de Cimento (t), dtype: float64

In [23]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Alagoas - Desemprego,Alagoas - IDH Educacao,Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado,Alagoas - IDH Longevidade,Alagoas - IDH Renda,Alagoas - IDH,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,0.540735,1.153503,0.54759,-0.669467,0.504915,0.391992,0.441591,0.935129,0.989474,1.108405,...,0.964765,-1.482779,-2.260447,-1.676368,-0.895465,-0.283979,1.831602,0.665102,-1.743991,0.61839


In [24]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[ 0.54073493,  1.1535032 ,  0.54759047, -0.66946663,
          0.50491453,  0.39199185,  0.44159091,  0.93512883,
          0.98947436,  1.10840491,  2.63582309, -0.18995911,
          0.72640489,  2.66833308, -0.032632  ,  0.98802435,
          0.24980494, -0.08955808,  1.85434394,  2.02706568,
         -2.21150703,  1.42938591,  0.85132668,  0.14092386,
          0.30902324,  1.00267904,  2.27970137,  1.11272593,
          0.61007902,  0.89064538,  1.42231438,  1.02650562,
          1.47127593, -1.53201473,  1.73503858,  1.66610613,
          1.0456894 ,  0.55289259, -1.81216593, -2.10342296,
         -2.55377478,  0.41940628, -1.75756445,  0.241023  ,
         -0.98659345,  1.52184916,  1.01398607,  1.49290967,
          1.40764092,  1.87930411, -2.51892072,  2.83356214,
          1.58620924, -0.02524128,  1.54231542,  1.5176119 ,
          1.7355269 ,  1.87941982,  1.54198129,  1.51276712,
          1.83795594,  2.62210693,  1.04993174,  0.96476493,
         -1.48277861, -2

In [25]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    576.578
Name: Alagoas - Consumo de Cimento (t), dtype: float64

In [26]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        tf.keras.utils.set_random_seed(seed)

    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=750, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True)
    
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(148, activation='tanh', return_sequences=True)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [27]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['loss'][500:])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [28]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[1120860268, 1296737998, 1250811715, 2278422922, 1393088938, 2242470807, 2108537623, 3833703379, 3351911274, 1476413141, 3105598143, 3976913308, 3916955361, 3592560670, 2875801470, 1654146845, 1907800743, 421834494, 1519376451, 4095078461, 2623907719, 815821268, 519524711, 2197571810, 2570489829, 3845732810, 390871128, 2585796915, 811757040, 481405567, 1113482108, 3065630351, 3522907679, 2678244886, 1219192598, 2791441259, 771366797, 2573638663, 842959235, 1625533861, 170954848, 47888610, 4056880532, 3756067705, 2294467085, 983694417, 2990357014, 2995822998, 2061550395, 1591910931, 3581058327, 2772782048, 1765298930, 2887407992, 1147054452, 3834663932, 3697714609, 1729848285, 1833551618, 1382078707, 54016759, 2349356211, 3025293566, 2481943259, 2110712850, 1252262360, 3784730608, 4245146742, 2696322022, 3471359169, 847458862, 1546137059, 3626327269, 372098583, 2352017806, 2685512544, 643496369, 943183034, 2397474409, 3271472832, 830862283, 844724874, 2819924824, 3768866942, 3421924978,

2023-09-12 07:02:51.048884: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-12 07:02:51.276658: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-12 07:02:51.276826: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

loss: 121.6226806640625
winner_seed: 1120860268


Step: 1 ___________________________________________
loss: 146.18943786621094


Step: 2 ___________________________________________
loss: 133.56858825683594


Step: 3 ___________________________________________
loss: 80.9820327758789
winner_seed: 2278422922


Step: 4 ___________________________________________
loss: 111.63794708251953


Step: 5 ___________________________________________
loss: 114.73355102539062


Step: 6 ___________________________________________
loss: 97.31610107421875


Step: 7 ___________________________________________
loss: 80.47492218017578
winner_seed: 3833703379


Step: 8 ___________________________________________
loss: 225.28704833984375


Step: 9 ___________________________________________
loss: 65.4759292602539
winner_seed: 1476413141


Step: 10 ___________________________________________
loss: 190.9318389892578


Step: 11 ___________________________________________
loss: 179.4469757080078


Step: 12 ______

In [29]:
trained_model, history = bidirectional_lstm_model(reshaped_train_input, 
                                                  train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 3s 3s/step - loss: 384489.1562
Epoch 2/10000
1/1 [==============================] - 0s 7ms/step - loss: 375963.5938
Epoch 3/10000
1/1 [==============================] - 0s 7ms/step - loss: 352894.9062
Epoch 4/10000
1/1 [==============================] - 0s 8ms/step - loss: 205864.1719
Epoch 5/10000
1/1 [==============================] - 0s 8ms/step - loss: 26597.2344
Epoch 6/10000
1/1 [==============================] - 0s 7ms/step - loss: 22232.2344
Epoch 7/10000
1/1 [==============================] - 0s 7ms/step - loss: 20004.1973
Epoch 8/10000
1/1 [==============================] - 0s 7ms/step - loss: 14997.8779
Epoch 9/10000
1/1 [==============================] - 0s 8ms/step - loss: 14807.3281
Epoch 10/10000
1/1 [==============================] - 0s 7ms/step - loss: 10164.5947
Epoch 11/10000
1/1 [==============================] - 0s 4ms/step - loss: 12327.1602
Epoch 12/10000
1/1 [==============================] - 0s 4ms/step - los

1/1 [==============================] - 0s 5ms/step - loss: 1150.7267
Epoch 195/10000
1/1 [==============================] - 0s 4ms/step - loss: 1408.0967
Epoch 196/10000
1/1 [==============================] - 0s 4ms/step - loss: 1450.0465
Epoch 197/10000
1/1 [==============================] - 0s 4ms/step - loss: 1418.9111
Epoch 198/10000
1/1 [==============================] - 0s 4ms/step - loss: 1631.8822
Epoch 199/10000
1/1 [==============================] - 0s 5ms/step - loss: 644.7518
Epoch 200/10000
1/1 [==============================] - 0s 4ms/step - loss: 912.2652
Epoch 201/10000
1/1 [==============================] - 0s 4ms/step - loss: 1179.4523
Epoch 202/10000
1/1 [==============================] - 0s 5ms/step - loss: 999.6292
Epoch 203/10000
1/1 [==============================] - 0s 5ms/step - loss: 2640.3127
Epoch 204/10000
1/1 [==============================] - 0s 4ms/step - loss: 2840.0784
Epoch 205/10000
1/1 [==============================] - 0s 5ms/step - loss: 1087.4586

1/1 [==============================] - 0s 4ms/step - loss: 722.3509
Epoch 292/10000
1/1 [==============================] - 0s 4ms/step - loss: 1042.2266
Epoch 293/10000
1/1 [==============================] - 0s 4ms/step - loss: 993.5574
Epoch 294/10000
1/1 [==============================] - 0s 4ms/step - loss: 1354.1752
Epoch 295/10000
1/1 [==============================] - 0s 5ms/step - loss: 1252.3472
Epoch 296/10000
1/1 [==============================] - 0s 5ms/step - loss: 1255.1085
Epoch 297/10000
1/1 [==============================] - 0s 5ms/step - loss: 641.1022
Epoch 298/10000
1/1 [==============================] - 0s 4ms/step - loss: 839.6504
Epoch 299/10000
1/1 [==============================] - 0s 5ms/step - loss: 1038.8663
Epoch 300/10000
1/1 [==============================] - 0s 5ms/step - loss: 1214.5643
Epoch 301/10000
1/1 [==============================] - 0s 4ms/step - loss: 2649.6870
Epoch 302/10000
1/1 [==============================] - 0s 5ms/step - loss: 1232.0195


1/1 [==============================] - 0s 4ms/step - loss: 2338.4167
Epoch 389/10000
1/1 [==============================] - 0s 4ms/step - loss: 1506.3861
Epoch 390/10000
1/1 [==============================] - 0s 4ms/step - loss: 852.0630
Epoch 391/10000
1/1 [==============================] - 0s 4ms/step - loss: 787.8108
Epoch 392/10000
1/1 [==============================] - 0s 4ms/step - loss: 2800.1592
Epoch 393/10000
1/1 [==============================] - 0s 5ms/step - loss: 2225.0713
Epoch 394/10000
1/1 [==============================] - 0s 4ms/step - loss: 819.1939
Epoch 395/10000
1/1 [==============================] - 0s 4ms/step - loss: 586.4673
Epoch 396/10000
1/1 [==============================] - 0s 5ms/step - loss: 1291.5820
Epoch 397/10000
1/1 [==============================] - 0s 5ms/step - loss: 676.0696
Epoch 398/10000
1/1 [==============================] - 0s 5ms/step - loss: 1847.7621
Epoch 399/10000
1/1 [==============================] - 0s 5ms/step - loss: 606.5283
Ep

1/1 [==============================] - 0s 5ms/step - loss: 1151.8964
Epoch 486/10000
1/1 [==============================] - 0s 4ms/step - loss: 1026.4788
Epoch 487/10000
1/1 [==============================] - 0s 4ms/step - loss: 1167.1652
Epoch 488/10000
1/1 [==============================] - 0s 5ms/step - loss: 430.7308
Epoch 489/10000
1/1 [==============================] - 0s 4ms/step - loss: 360.5248
Epoch 490/10000
1/1 [==============================] - 0s 4ms/step - loss: 870.8369
Epoch 491/10000
1/1 [==============================] - 0s 5ms/step - loss: 1045.2502
Epoch 492/10000
1/1 [==============================] - 0s 4ms/step - loss: 1355.4469
Epoch 493/10000
1/1 [==============================] - 0s 4ms/step - loss: 871.2438
Epoch 494/10000
1/1 [==============================] - 0s 4ms/step - loss: 357.6190
Epoch 495/10000
1/1 [==============================] - 0s 5ms/step - loss: 793.7147
Epoch 496/10000
1/1 [==============================] - 0s 4ms/step - loss: 649.5719
Epo

...

1/1 [==============================] - 0s 5ms/step - loss: 177.3438
Epoch 2123/10000
1/1 [==============================] - 0s 4ms/step - loss: 749.5518
Epoch 2124/10000
1/1 [==============================] - 0s 4ms/step - loss: 281.6096
Epoch 2125/10000
1/1 [==============================] - 0s 4ms/step - loss: 457.1660
Epoch 2126/10000
1/1 [==============================] - 0s 5ms/step - loss: 624.4769
Epoch 2127/10000
1/1 [==============================] - 0s 5ms/step - loss: 640.0695
Epoch 2128/10000
1/1 [==============================] - 0s 5ms/step - loss: 923.9614
Epoch 2129/10000
1/1 [==============================] - 0s 4ms/step - loss: 702.5203
Epoch 2130/10000
1/1 [==============================] - 0s 4ms/step - loss: 302.3273
Epoch 2131/10000
1/1 [==============================] - 0s 4ms/step - loss: 352.1443
Epoch 2132/10000
1/1 [==============================] - 0s 5ms/step - loss: 423.7968
Epoch 2133/10000
1/1 [==============================] - 0s 5ms/step - loss: 812.90

1/1 [==============================] - 0s 6ms/step - loss: 847.4103
Epoch 2315/10000
1/1 [==============================] - 0s 6ms/step - loss: 796.5147
Epoch 2316/10000
1/1 [==============================] - 0s 4ms/step - loss: 913.2598
Epoch 2317/10000
1/1 [==============================] - 0s 6ms/step - loss: 528.7457
Epoch 2318/10000
1/1 [==============================] - 0s 6ms/step - loss: 846.9819
Epoch 2319/10000
1/1 [==============================] - 0s 6ms/step - loss: 1110.6327
Epoch 2320/10000
1/1 [==============================] - 0s 6ms/step - loss: 695.4258
Epoch 2321/10000
1/1 [==============================] - 0s 4ms/step - loss: 613.1013
Epoch 2322/10000
1/1 [==============================] - 0s 5ms/step - loss: 1261.9523
Epoch 2323/10000
1/1 [==============================] - 0s 6ms/step - loss: 948.0249
Epoch 2324/10000
1/1 [==============================] - 0s 5ms/step - loss: 735.2507
Epoch 2325/10000
1/1 [==============================] - 0s 5ms/step - loss: 639.

1/1 [==============================] - 0s 4ms/step - loss: 921.6462
Epoch 2507/10000
1/1 [==============================] - 0s 5ms/step - loss: 1645.6117
Epoch 2508/10000
1/1 [==============================] - 0s 4ms/step - loss: 606.2903
Epoch 2509/10000
1/1 [==============================] - 0s 5ms/step - loss: 645.1780
Epoch 2510/10000
1/1 [==============================] - 0s 5ms/step - loss: 765.3981
Epoch 2511/10000
1/1 [==============================] - 0s 6ms/step - loss: 1229.8221
Epoch 2512/10000
1/1 [==============================] - 0s 5ms/step - loss: 312.4118
Epoch 2513/10000
1/1 [==============================] - 0s 5ms/step - loss: 766.7949
Epoch 2514/10000
1/1 [==============================] - 0s 6ms/step - loss: 412.4754
Epoch 2515/10000
1/1 [==============================] - 0s 4ms/step - loss: 309.1149
Epoch 2516/10000
1/1 [==============================] - 0s 4ms/step - loss: 571.5665
Epoch 2517/10000
1/1 [==============================] - 0s 5ms/step - loss: 612.

1/1 [==============================] - 0s 5ms/step - loss: 535.5859
Epoch 2699/10000
1/1 [==============================] - 0s 5ms/step - loss: 603.6187
Epoch 2700/10000
1/1 [==============================] - 0s 5ms/step - loss: 368.6026
Epoch 2701/10000
1/1 [==============================] - 0s 5ms/step - loss: 1649.3604
Epoch 2702/10000
1/1 [==============================] - 0s 5ms/step - loss: 873.9205
Epoch 2703/10000
1/1 [==============================] - 0s 5ms/step - loss: 530.0083
Epoch 2704/10000
1/1 [==============================] - 0s 5ms/step - loss: 566.2948
Epoch 2705/10000
1/1 [==============================] - 0s 5ms/step - loss: 764.7023
Epoch 2706/10000
1/1 [==============================] - 0s 5ms/step - loss: 1014.9464
Epoch 2707/10000
1/1 [==============================] - 0s 5ms/step - loss: 366.8054
Epoch 2708/10000
1/1 [==============================] - 0s 5ms/step - loss: 670.9627
Epoch 2709/10000
1/1 [==============================] - 0s 5ms/step - loss: 509.

1/1 [==============================] - 0s 5ms/step - loss: 356.6538
Epoch 2891/10000
1/1 [==============================] - 0s 5ms/step - loss: 615.1928
Epoch 2892/10000
1/1 [==============================] - 0s 5ms/step - loss: 604.9195
Epoch 2893/10000
1/1 [==============================] - 0s 5ms/step - loss: 642.0666
Epoch 2894/10000
1/1 [==============================] - 0s 7ms/step - loss: 396.4239
Epoch 2895/10000
1/1 [==============================] - 0s 6ms/step - loss: 554.8135
Epoch 2896/10000
1/1 [==============================] - 0s 4ms/step - loss: 506.1571
Epoch 2897/10000
1/1 [==============================] - 0s 4ms/step - loss: 975.2459
Epoch 2898/10000
1/1 [==============================] - 0s 4ms/step - loss: 1362.2839
Epoch 2899/10000
1/1 [==============================] - 0s 6ms/step - loss: 832.4193
Epoch 2900/10000
1/1 [==============================] - 0s 6ms/step - loss: 766.0703
Epoch 2901/10000
1/1 [==============================] - 0s 6ms/step - loss: 978.6

In [30]:
prediction = trained_model.predict(reshaped_test_input)[0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 1s 588ms/step
[576.578](test_target) - [[579.60223]](prediction) = 3.0242338867187755


In [31]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.00524514269833184

In [32]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = bidirectional_lstm_model(dfToInputRNN(train_input), 
                                                     train_target, want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [33]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Alagoas - Desemprego,Alagoas - IDH Educacao,Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado,Alagoas - IDH Longevidade,Alagoas - IDH Renda,Alagoas - Produção de Cimento (t),Alagoas - IDH,...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-0.592143,-1.238768,-0.940263,-0.691783,-0.837853,-0.796510,-1.270151,-1.293364,-1.223483,-1.255860,...,1.277766,1.308949,1.412346,-0.914375,1.380911,1.411042,1.341066,-1.336272,-1.411478,1.406725
1,1.408288,0.028544,-0.444707,1.414104,-0.567735,-0.613762,0.096525,0.151296,-0.002520,0.064802,...,-0.114016,-0.190804,-0.643250,1.391499,-0.954679,-0.623537,-0.281746,1.069113,0.629592,-0.577490
2,-0.816144,1.210223,1.384970,-0.722321,1.405588,1.410272,1.173626,1.142068,1.226003,1.191057,...,-1.163750,-1.118144,-0.769096,-0.477124,-0.426232,-0.787505,-1.059321,0.267160,0.781885,-0.829235


train_target:


,Alagoas - Consumo de Cimento (t)
0,471.072
1,583.772
2,742.881


test_input:


...

,Alagoas - Desemprego,Alagoas - IDH Educacao,Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado,Alagoas - IDH Longevidade,Alagoas - IDH Renda,Alagoas - Produção de Cimento (t),Alagoas - IDH,...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,0.540735,1.153503,0.54759,-0.669467,0.504915,0.391992,0.441591,0.935129,-1.056172,0.989474,...,0.964765,-1.482779,-2.260447,-1.676368,-0.895465,-0.283979,1.831602,0.665102,-1.743991,0.61839


test_target:


,Alagoas - Consumo de Cimento (t)
12,576.578


1/1 [==============================] - 1s 574ms/step
Error: 10.397830078125025




[747.1240844726562,
 836.0765991210938,
 599.31103515625,
 800.3223876953125,
 660.7962646484375,
 548.29638671875,
 377.911376953125,
 336.822998046875,
 388.02825927734375,
 586.975830078125]

In [34]:
display(targets)
display(predictions)

[844.864,
 791.375,
 818.6451428571429,
 676.3001142857142,
 538.5867428571428,
 362.791,
 340.278,
 395.359,
 581.479,
 576.578]

[747.1240844726562,
 836.0765991210938,
 599.31103515625,
 800.3223876953125,
 660.7962646484375,
 548.29638671875,
 377.911376953125,
 336.822998046875,
 388.02825927734375,
 586.975830078125]

In [35]:
mae = mean_absolute_error(predictions, targets)
mae

109.35307539760046

In [36]:
porcentage = mae/np.mean(targets)
porcentage

0.18452303680030102